In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cardiovascular_Disease"
cohort = "GSE182600"

# Input paths
in_trait_dir = "../../input/GEO/Cardiovascular_Disease"
in_cohort_dir = "../../input/GEO/Cardiovascular_Disease/GSE182600"

# Output paths
out_data_file = "../../output/preprocess/Cardiovascular_Disease/GSE182600.csv"
out_gene_data_file = "../../output/preprocess/Cardiovascular_Disease/gene_data/GSE182600.csv"
out_clinical_data_file = "../../output/preprocess/Cardiovascular_Disease/clinical_data/GSE182600.csv"
json_path = "../../output/preprocess/Cardiovascular_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene Expression of Cardiogenic Shock Patients under Extracorporeal Membrane Oxygenation"
!Series_summary	"Prognosis for cardiogenic shock patients under ECMO was our study goal. Success defined as survived more than 7 days after ECMO installation and failure died or had multiple organ failure in 7 days. Total 34 cases were enrolled, 17 success and 17 failure."
!Series_summary	"Peripheral blood mononuclear cells collected at ECMO installation 0hr, 2hr and removal were used analyzed."
!Series_overall_design	"Analysis of the cardiogenic shock patients at extracorporeal membrane oxygenation treatment by genome-wide gene expression. Transcriptomic profiling between successful and failure groups were analyzed."
Sample Characteristics Dictionary:
{0: ['disease state: Acute myocarditis', 'disease state: Acute myocardial infarction', 'disease state: Dilated cardiomyopathy, DCMP', 'disease state: Congestive heart failure', 'disease state: Dilated cardiomyop

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background info, this dataset likely contains gene expression data from PBMCs
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait: Cardiovascular disease outcome, available at key 3
trait_row = 3

# Age: Available at key 1
age_row = 1

# Gender: Available at key 2
gender_row = 2

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait values to binary (0 for success, 1 for failure)"""
    if value is None:
        return None
    
    # Extract value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.lower() == 'success':
        return 0
    elif value.lower() in ['failure', 'fail']:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if value is None:
        return None
    
    # Extract value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # First, we need to create a function to get feature data since we're not using the normal workflow
    def get_feature_data(clinical_dict, row_idx, feature_name, convert_func):
        if row_idx is None or row_idx not in clinical_dict:
            return pd.DataFrame()
        
        raw_values = clinical_dict[row_idx]
        processed_values = [convert_func(val) for val in raw_values]
        
        # Create a dataframe with sample IDs as columns and feature as rows
        feature_df = pd.DataFrame([processed_values], columns=[f"Sample_{i+1}" for i in range(len(processed_values))])
        feature_df.index = [feature_name]
        
        return feature_df
    
    # Sample characteristics dictionary from the previous step output
    sample_chars_dict = {
        0: ['disease state: Acute myocarditis', 'disease state: Acute myocardial infarction', 'disease state: Dilated cardiomyopathy, DCMP', 'disease state: Congestive heart failure', 'disease state: Dilated cardiomyopathy', 'disease state: Arrhythmia', 'disease state: Aortic dissection'],
        1: ['age: 33.4', 'age: 51.2', 'age: 51.9', 'age: 47.8', 'age: 41.5', 'age: 67.3', 'age: 52.8', 'age: 16.1', 'age: 78.9', 'age: 53.2', 'age: 70.9', 'age: 59.9', 'age: 21.9', 'age: 45.2', 'age: 52.4', 'age: 32.3', 'age: 55.8', 'age: 47', 'age: 57.3', 'age: 31.7', 'age: 49.3', 'age: 66.1', 'age: 55.9', 'age: 49.1', 'age: 63', 'age: 21', 'age: 53.6', 'age: 50.1', 'age: 37.4', 'age: 71.5'],
        2: ['gender: F', 'gender: M'],
        3: ['outcome: Success', 'outcome: Failure', 'outcome: failure'],
        4: ['cell type: PBMC'],
        5: ['time: 0hr', 'time: 2hr', 'time: Removal']
    }
    
    # Create feature DataFrames
    feature_list = []
    trait_data = get_feature_data(sample_chars_dict, trait_row, trait, convert_trait)
    feature_list.append(trait_data)
    
    if age_row is not None:
        age_data = get_feature_data(sample_chars_dict, age_row, 'Age', convert_age)
        feature_list.append(age_data)
    
    if gender_row is not None:
        gender_data = get_feature_data(sample_chars_dict, gender_row, 'Gender', convert_gender)
        feature_list.append(gender_data)
    
    # Combine all features
    selected_clinical_df = pd.concat(feature_list, axis=0)
    
    # Transpose the DataFrame to have samples as rows and features as columns
    selected_clinical_df = selected_clinical_df.transpose()
    
    # Preview the result
    print("Preview of extracted clinical features:")
    preview = preview_df(selected_clinical_df)
    print(preview)
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of extracted clinical features:
{'Cardiovascular_Disease': [0.0, 1.0, 1.0, nan, nan], 'Age': [33.4, 51.2, 51.9, 47.8, 41.5], 'Gender': [0.0, 1.0, nan, nan, nan]}
Clinical data saved to: ../../output/preprocess/Cardiovascular_Disease/clinical_data/GSE182600.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Cardiovascular_Disease/GSE182600/GSE182600_series_matrix.txt.gz


Gene data shape: (29363, 78)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, I can see they are ILMN_* format
# These are Illumina probe IDs (BeadArray technology), not human gene symbols
# They will need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'Transcript', 'Species', 'Source', 'Search_Key', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns to use for gene mapping
# From the preview, we can see that 'ID' contains the ILMN_* identifiers matching gene_data's index
# 'Symbol' contains the gene symbols we want to map to
print("\nIdentifying mapping columns:")
print(f"Probe ID column: 'ID'")
print(f"Gene Symbol column: 'Symbol'")

# 2. Create a gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Preview the mapping data
print("\nPreview of gene mapping data:")
print(preview_df(gene_mapping))
print(f"Gene mapping shape: {gene_mapping.shape}")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the results
print("\nPreview of resulting gene expression data:")
print(f"Gene data shape after mapping: {gene_data.shape}")
print(f"First 10 gene symbols: {gene_data.index[:10].tolist()}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to: {out_gene_data_file}")



Identifying mapping columns:
Probe ID column: 'ID'
Gene Symbol column: 'Symbol'

Preview of gene mapping data:
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144']}
Gene mapping shape: (29377, 2)



Preview of resulting gene expression data:
Gene data shape after mapping: (20206, 78)
First 10 gene symbols: ['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT']



Gene expression data saved to: ../../output/preprocess/Cardiovascular_Disease/gene_data/GSE182600.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data_normalized.shape}")
print(f"First 10 normalized gene symbols: {gene_data_normalized.index[:10].tolist()}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 2. Re-extract the clinical data directly from the source
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Define conversion functions based on previous analysis
def convert_trait(value):
    """Convert trait values to binary (0 for success, 1 for failure)"""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if it exists
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.lower() == 'success':
        return 0
    elif value.lower() in ['failure', 'fail']:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if it exists
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)"""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if it exists
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# Use the correct row indices from previous analysis
trait_row = 3  # outcome
age_row = 1    # age
gender_row = 2 # gender

# Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_data,
    trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save clinical features - make sure to transpose it so samples are rows, not columns
clinical_features_t = clinical_features.T  # Transpose for proper format
clinical_features_t.to_csv(out_clinical_data_file)
print(f"Clinical features saved to: {out_clinical_data_file}")
print("Clinical features preview (transposed):")
print(preview_df(clinical_features_t))

# 3. Link clinical and genetic data - use the transposed clinical data
linked_data = pd.merge(
    clinical_features_t, 
    gene_data_normalized.T,  # Transpose gene data so genes are columns
    left_index=True, 
    right_index=True,
    how='inner'
)

print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
preview_cols = min(5, linked_data.shape[1])
print(linked_data.iloc[:5, :preview_cols])

# 4. Handle missing values
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "Dataset contains gene expression data from PBMCs of cardiogenic shock patients under ECMO treatment, comparing successful and failed outcomes."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Gene data shape after normalization: (19445, 78)
First 10 normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Normalized gene data saved to: ../../output/preprocess/Cardiovascular_Disease/gene_data/GSE182600.csv
Clinical features saved to: ../../output/preprocess/Cardiovascular_Disease/clinical_data/GSE182600.csv
Clinical features preview (transposed):
{'Cardiovascular_Disease': [0.0, 0.0, 1.0, 0.0, 1.0], 'Age': [33.4, 51.2, 51.9, 47.8, 41.5], 'Gender': [0.0, 1.0, 0.0, 1.0, 0.0]}
Linked data shape: (78, 19448)
Linked data preview (first 5 rows, 5 columns):
            Cardiovascular_Disease   Age  Gender        A1BG     A1BG-AS1
GSM5532093                     0.0  33.4     0.0  123.145500  1284.286536
GSM5532094                     0.0  51.2     1.0  134.323626  2123.843378
GSM5532095                     1.0  51.9     0.0  100.294706  1088.857429
GSM5532096                     0.0  47.8     1.0  130.315854  1074.517347
GSM5532097                     1.0  41.5     0.0  106.890941  1070.809003


Linked data shape after handling missing values: (78, 19448)
For the feature 'Cardiovascular_Disease', the least common label is '1.0' with 31 occurrences. This represents 39.74% of the dataset.
The distribution of the feature 'Cardiovascular_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 47.0
  50% (Median): 52.15
  75%: 56.35
Min: 16.1
Max: 78.9
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 24 occurrences. This represents 30.77% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Cardiovascular_Disease/cohort_info.json


Linked data saved to ../../output/preprocess/Cardiovascular_Disease/GSE182600.csv
